# Re-evaluate functions and compare
* Previous optimizations over other seeds
* Check if Octave underperforms

## Function minimum

In [1]:
from skopt import load
def objective():
    pass
mult = [load("./optims/scenario3/gbrt_" + str(seed) + "_100.opt") for seed in range(5)]
best = mult[4]
best.x

[0.00010334762307546188,
 1.5415805242432643e-05,
 14.519110660781786,
 93,
 6.1054043480560045,
 40]

In [2]:
import numpy as np
best.x_iters[np.argmin(best.func_vals)]

[0.00010334762307546188,
 1.5415805242432643e-05,
 14.519110660781786,
 93,
 6.1054043480560045,
 40]

## Cumulative function optimum

In [3]:
from optimize import res_optimum, res_optimum_mult


Starting MATLAB engine...
-------------------------
Elapsed: 32.11 sec


In [4]:
res_optimum(best)

([0.0002109673047643446,
  0.05723377028423132,
  409.1369490061136,
  170,
  0.9280712004527215,
  93],
 0.5977239534927569)

In [5]:
res_optimum_mult(mult)

([0.0010898482519426577,
  0.0039172298798781716,
  2.547687882806528,
  24,
  506.27253684590727,
  174],
 0.5484916099100569)

## Evaluate Minimum

In [ ]:
import full_model
from skopt import load
from scipy.io import loadmat
from math import log10

function = full_model.run_model
dataset = "Coil20.mat"
ep_prtr = 1001
ep = 251
arg_names = ["lr_pretrain", "lr", "alpha1", "maxIter1", "lambda1", "lambda2", "alpha2", "maxIter2"]
arg_vals = load("./optims/scenario4/gp_3_140.opt")

args = dict(zip(arg_names, arg_vals))
data_loaded = loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

total = 1.0
for seed in range(100, 105):
    print(seed)
    value = 1-function(images_norm, labels, seed, ep_prtr, ep, **args, verbose=False)[0]
    print("value:", value)
    total *= value

print("AVERAGE:", total**0.2)

function = full_model.run_ssc
dataset = "yaleB.mat"
arg_names = ["alpha1", "maxIter1"]
arg_vals = load("./optims/scenario5/gp_2_50.opt")

args = dict(zip(arg_names, arg_vals))
data_loaded = loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

total = 1.0
for seed in range(100, 105):
    print(seed)
    value = 1-function(images_norm, labels, seed, ep_prtr, ep, **args, verbose=False)[0]
    print("value:", value)
    total *= value

print("AVERAGE:", total**0.2)

function = full_model.run_ssc
dataset = "Coil20.mat"
arg_names = ["alpha1", "maxIter1"]
arg_vals = load("./optims/scenario6/forest_3_50.opt")

args = dict(zip(arg_names, arg_vals))
data_loaded = loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

total = 1.0
for seed in range(100, 105):
    print(seed)
    value = 1-function(images_norm, labels, seed, ep_prtr, ep, **args, verbose=False)[0]
    print("value:", value)
    total *= value

print("AVERAGE:", total**0.2)

function = full_model.run_ae
dataset = "yaleB.mat"
ep_prtr = 201
ep = 101
arg_names = ["lr_pretrain", "lr", "lambda1", "lambda2", "alpha2", "maxIter2"]
arg_vals = load("./optims/scenario7/gp_0_50.opt")

args = dict(zip(arg_names, arg_vals))
data_loaded = loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

total = 1.0
for seed in range(100, 105):
    print(seed)
    value = 1-function(images_norm, labels, seed, ep_prtr, ep, **args, verbose=False)[0]
    print("value:", value)
    total *= value

print("AVERAGE:", total**0.2)

function = full_model.run_ae
dataset = "Coil20.mat"
ep_prtr = 1001
ep = 251
arg_names = ["lr_pretrain", "lr", "lambda1", "lambda2", "alpha2", "maxIter2"]
arg_vals = load("./optims/scenario8/forest_0_50.opt")

args = dict(zip(arg_names, arg_vals))
data_loaded = loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

total = 1.0
for seed in range(100, 105):
    print(seed)
    value = 1-function(images_norm, labels, seed, ep_prtr, ep, **args, verbose=False)[0]
    print("value:", value)
    total *= value

print("AVERAGE:", total**0.2)

## Evaluate Optimum

In [10]:
scenario = 2
best_opt = "forest"
max_iters = 50
optimum = res_optimum_mult([load("./optims/scenario"+str(scenario)+"/"+best_opt+"_"+str(seed)+"_"+str(max_iters)+".opt") for seed in range(5)])
print("optimum:")
print(optimum)

function = full_model.run_model
dataset = "Coil20.mat"
ep_prtr = 201
ep = 101
arg_names = ["lr_pretrain", "lr", "alpha1", "maxIter1", "lambda1", "lambda2", "alpha2", "maxIter2"]
arg_vals = optimum[0]

args = dict(zip(arg_names, arg_vals))
data_loaded = loadmat("./saved/processed/" + dataset)
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

total = 1.0
for seed in range(100, 105):
    print(seed)
    value = 1-function(images_norm, labels, seed, ep_prtr, ep, **args, verbose=False)[0]
    print("value:", value)
    total *= value

print("AVERAGE:", total**0.2)

optimum:
([0.005723346377343881, 0.0006960540795460365, 1.0686885283416514, 85, 2.527546354637533, 100], 0.725669763942158)
100
value: 0.9493055555555555
101
value: 0.9493055555555555
102
value: 0.9493055555555555
103
value: 0.9493055555555555
104
value: 0.9493055555555555
AVERAGE: 0.9493055555555555
